<a href="https://colab.research.google.com/github/hrytsiukp/DSC/blob/master/linear_regression_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**МОДЕЛЬ ЛІНІЙНОЇ РЕГРЕСІЇ**

In [1]:
# Імпорт бібліотек та модулів
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
# Встановлення випадкових параметрів для відтворюваності результатів
np.random.seed(80)


In [2]:
# Завантаження даних з диска
from google.colab import files

uploaded = files.upload() # Зчитування файлу з диска. Після зчитування цей рядок задокументувати
file = "region.csv" # для Google Colab

XY = pd.read_csv(file)
XY.info() # інформація про дані


Saving region.csv to region.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   t1      110 non-null    float64
 1   t2      110 non-null    float64
 2   t3      110 non-null    float64
 3   t4      110 non-null    float64
 4   t5      110 non-null    float64
 5   t6      110 non-null    float64
 6   t7      110 non-null    float64
 7   t8      110 non-null    float64
 8   t9      110 non-null    float64
 9   eps     110 non-null    float64
dtypes: float64(10)
memory usage: 8.7 KB


In [3]:
# Видалення 5 зразків згідно з варіантом

variant = 1
rows_to_drop = np.arange(variant, XY.shape[0], 22)  # кожен 22-й рядок
XY = XY.drop(index=rows_to_drop, errors='ignore').reset_index(drop=True)



In [4]:
# Розділення змінних на впливаючі фактори (фічі) та змінну -відгук
import statsmodels.api as sm
# Визначення факторів (всі колонки, крім колонки з змінною-відгуком eps)
X = XY.drop(columns=['eps'])
# Додати константу до матриці X
X = sm.add_constant(X)
# Визначення залежної змінної
y = XY['eps']


In [5]:
# Побудова загальної моделі з усіма факторами
model = sm.OLS(y, X).fit()
print(model.summary())
#  Якщо модель має високе значення Cond. No., це вказує на низьку якість моделі. Для підвищення якості моделі потрібно видалити незначущі фактори


                            OLS Regression Results                            
Dep. Variable:                    eps   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     28.39
Date:                Sun, 08 Feb 2026   Prob (F-statistic):           2.96e-23
Time:                        18:58:45   Log-Likelihood:                -249.50
No. Observations:                 105   AIC:                             519.0
Df Residuals:                      95   BIC:                             545.5
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         29.4615      5.704      5.165      0.0

In [15]:
# Метод кросс-валідації
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

# Визначення кількості складок
k_folds = 5
# Ініціалізація об'єкту KFold для розділення даних на складки
kf = KFold(n_splits=k_folds)
# kf = KFold(n_splits=k_folds, shuffle=True, random_state=80)

rows = []
# Створення списків для збереження результатів
# mse_scores = []
# rmse_scores = []
# r2_scores = []


In [17]:
# Виконання K-fold Cross-Validation
for i, (train_index, test_index) in enumerate(kf.split(X), 1):
# for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

     # Навчання моделі на навчальних даних
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Передбачення на тестових даних
    y_pred = model.predict(X_test)

    # Обчислення середньоквадратичної помилки
    mse = mean_squared_error(y_test, y_pred)
    # універсальна помилка
    rmse = np.sqrt(mse)
    # Обчислення R-квадрат
    r2 = r2_score(y_test, y_pred)

    rows.append({
        "Fold": i,
        "Train n": len(train_index),
        "Test n": len(test_index),
        "MSE": mse,
        "RMSE": rmse,
        "R2": r2
    })




K-Fold results:


,Fold,Train n,Test n,MSE,RMSE,R2
0,1,84,21,4.471,2.114,0.817
1,2,84,21,6.217,2.493,0.735
2,3,84,21,12.107,3.480,0.441
3,4,84,21,10.524,3.244,0.643
4,5,84,21,5.453,2.335,0.782



Average over folds:


,MSE,RMSE,R2
Mean,7.754,2.733,0.684


In [18]:
# Таблиця результатів по складках
cv_table = pd.DataFrame(rows)

# оформлення таблиці
cv_table_show = cv_table.copy()
cv_table_show[["MSE", "RMSE", "R2"]] = cv_table_show[["MSE", "RMSE", "R2"]].round(3)

print("K-Fold results:")
display(cv_table_show)

# Підсумок : середні значення
summary = cv_table[["MSE", "RMSE", "R2"]].mean().to_frame(name="Mean").T
summary = summary.round(3)
print("\nAverage over folds:")
display(summary)

K-Fold results:


,Fold,Train n,Test n,MSE,RMSE,R2
0,1,84,21,4.471,2.114,0.817
1,2,84,21,6.217,2.493,0.735
2,3,84,21,12.107,3.480,0.441
3,4,84,21,10.524,3.244,0.643
4,5,84,21,5.453,2.335,0.782



Average over folds:


,MSE,RMSE,R2
Mean,7.754,2.733,0.684
